## Package

In [4]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from huggingface_hub import login
import json
import os
import ollama
import time
from datetime import datetime
from tqdm import tqdm
import re

/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## Get Data

In [ ]:
# 資料讀取選項
# 您可以選擇以下任一種方式來載入資料：

# 選項 1: 從已儲存的本地檔案讀取 (推薦，速度快)
use_local_files = False

# 選項 2: 從 Hugging Face 直接串流載入 (需要網路連線)
use_streaming = True

# 選項 3: 下載完整資料集 (檔案很大，不推薦)
use_full_download = False

print("=== 資料載入選項 ===")
print(f"使用本地檔案: {use_local_files}")
print(f"使用串流模式: {use_streaming}")
print(f"下載完整資料集: {use_full_download}")

# 資料載入
if use_local_files:
    print("\n📁 從本地檔案讀取資料...")
    
    # 檢查已儲存的檔案
    save_dir = "saved_datasets"
    
    if os.path.exists(save_dir):
        import glob
        
        # 尋找可用的檔案
        csv_files = glob.glob(f"{save_dir}/*.csv")
        json_files = glob.glob(f"{save_dir}/*.json")
        parquet_files = glob.glob(f"{save_dir}/*.parquet")
        
        print(f"找到的檔案:")
        print(f"  CSV 檔案: {len(csv_files)} 個")
        print(f"  JSON 檔案: {len(json_files)} 個")
        print(f"  Parquet 檔案: {len(parquet_files)} 個")
        
        # 優先使用 Parquet 檔案 (最高效)
        if parquet_files:
            latest_file = max(parquet_files, key=os.path.getctime)
            print(f"\n📊 讀取最新的 Parquet 檔案: {latest_file}")
            df = pd.read_parquet(latest_file)
            
        # 其次使用 CSV 檔案
        elif csv_files:
            latest_file = max(csv_files, key=os.path.getctime)
            print(f"\n📊 讀取最新的 CSV 檔案: {latest_file}")
            df = pd.read_csv(latest_file)
            
        # 最後使用 JSON 檔案
        elif json_files:
            latest_file = max(json_files, key=os.path.getctime)
            print(f"\n📊 讀取最新的 JSON 檔案: {latest_file}")
            with open(latest_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            df = pd.DataFrame(data)
            
        else:
            print("❌ 沒有找到已儲存的資料檔案")
            print("請先執行資料下載和儲存的程式碼")
            df = None
    else:
        print("❌ 找不到 saved_datasets 目錄")
        print("請先執行資料下載和儲存的程式碼")
        df = None

elif use_streaming:
    print("\n🌐 從 Hugging Face 串流載入資料...")
    
    # 使用串流模式載入資料集
    dataset = load_dataset("austenjs/ClueCorpusSmallDataset", streaming=True)
    
    # 設定要載入的樣本數量 - 減少到100筆用於演示
    num_samples = 100
    print(f"載入前 {num_samples} 筆資料...")
    
    # 收集資料
    sample_data = []
    for i, example in enumerate(dataset['train']):
        if i >= num_samples:
            break
        sample_data.append(example)
        if (i + 1) % 25 == 0:
            print(f"  已載入 {i + 1} 筆資料...")
    
    # 轉換為 DataFrame
    df = pd.DataFrame(sample_data)
    
elif use_full_download:
    print("\n⬇️ 下載完整資料集...")
    print("警告：這將下載 13.7GB 的資料，可能需要很長時間")
    
    # 下載完整資料集
    dataset = load_dataset("austenjs/ClueCorpusSmallDataset")
    df = dataset['train'].to_pandas()

else:
    print("❌ 沒有選擇任何資料載入選項")
    df = None

# 顯示資料資訊
if df is not None:
    print(f"\n✅ 資料載入成功！")
    print(f"📊 資料形狀: {df.shape}")
    print(f"📋 欄位名稱: {list(df.columns)}")
    
    # 顯示基本統計
    if 'text' in df.columns: # type: ignore
        df['text_length'] = df['text'].str.len() # type: ignore
        print(f"\n📈 文本長度統計:")
        print(df['text_length'].describe()) # type: ignore
        
        # 顯示前幾筆資料範例
        print(f"\n📝 前 3 筆資料範例:")
        for i in range(min(3, len(df))): # type: ignore
            text = df.iloc[i]['text']
            # 顯示前100個字符
            preview = text[:100] + "..." if len(text) > 100 else text
            print(f"範例 {i+1} ({len(text)} 字符): {preview}")
            print("-" * 80)
    
    print(f"\n🎯 資料已準備就緒，可用於後續的 LLM 評分處理！")
else:
    print("\n❌ 資料載入失敗，請檢查設定並重新執行")

# 儲存到全域變數供後續使用
globals()['dataset_df'] = df

=== 資料載入選項 ===
使用本地檔案: True
使用串流模式: False
下載完整資料集: False

📁 從本地檔案讀取資料...
找到的檔案:
  CSV 檔案: 1 個
  JSON 檔案: 1 個
  Parquet 檔案: 1 個

📊 讀取最新的 Parquet 檔案: saved_datasets/clue_corpus_small_20250901_085816.parquet

✅ 資料載入成功！
📊 資料形狀: (1000, 1)
📋 欄位名稱: ['text']

📈 文本長度統計:
count     1000.000000
mean       300.899000
std        785.763282
min          5.000000
25%         38.000000
50%        105.500000
75%        274.000000
max      17020.000000
Name: text_length, dtype: float64

📝 前 3 筆資料範例:
範例 1 (132 字符): 130真是佩服这家店开这么久。尽管门面已经小了一圈，但还是开着不容易啊。我们不容易，老板也不容易。自助餐，你可以吃得比平时多，但决不能浪费。想吃回20元，那是不可能的，所以还是不要去了。菜真的很一般，...
--------------------------------------------------------------------------------
範例 2 (8 字符): 送货速度奇慢无比
--------------------------------------------------------------------------------
範例 3 (12 字符): 这是自己用过最好的用的了
--------------------------------------------------------------------------------

🎯 資料已準備就緒，可用於後續的 LLM 評分處理！


## 📝 文本切分處理

In [6]:
# 📝 文本切分處理 - 切分為句子級別的片段
print("🔪 啟動文本切分處理...")

def split_text_to_sentences(text, min_length=10, max_length=50):
    """
    將文本切分為句子級別的片段
    
    Args:
        text (str): 原始文本
        min_length (int): 最小片段長度
        max_length (int): 最大片段長度
    
    Returns:
        list: 切分後的句子片段列表
    """
    # 定義標點符號分隔符
    sentence_separators = ['。', '！', '？', '；', '…']  # 強句號分隔符
    phrase_separators = ['，', '、', '：', '；']  # 弱分隔符
    
    # 1. 首先按強標點符號切分成句子
    sentences = []
    current_sentence = ""
    
    for char in text:
        current_sentence += char
        if char in sentence_separators:
            if current_sentence.strip():
                sentences.append(current_sentence.strip())
                current_sentence = ""
    
    # 處理最後一個句子（如果沒有以強標點結尾）
    if current_sentence.strip():
        sentences.append(current_sentence.strip())
    
    # 2. 對每個句子進一步按逗號等分隔符切分
    fragments = []
    
    for sentence in sentences:
        # 跳過太短的句子
        if len(sentence) < min_length:
            continue
            
        # 如果句子長度在合理範圍內，直接使用
        if len(sentence) <= max_length:
            fragments.append(sentence)
        else:
            # 對長句子按逗號等進一步切分
            parts = []
            current_part = ""
            
            for char in sentence:
                current_part += char
                if char in phrase_separators:
                    if current_part.strip() and len(current_part.strip()) >= min_length:
                        parts.append(current_part.strip())
                        current_part = ""
            
            # 處理最後一部分
            if current_part.strip() and len(current_part.strip()) >= min_length:
                parts.append(current_part.strip())
            
            # 如果切分後的部分太短，嘗試合併
            merged_parts = []
            temp_part = ""
            
            for part in parts:
                if len(temp_part + part) <= max_length:
                    temp_part = temp_part + part if temp_part else part
                else:
                    if temp_part and len(temp_part) >= min_length:
                        merged_parts.append(temp_part)
                    temp_part = part
            
            # 添加最後一部分
            if temp_part and len(temp_part) >= min_length:
                merged_parts.append(temp_part)
            
            # 如果切分失敗，直接截斷
            if not merged_parts and len(sentence) >= min_length:
                # 簡單截斷成合適長度的片段
                for i in range(0, len(sentence), max_length):
                    fragment = sentence[i:i+max_length]
                    if len(fragment) >= min_length:
                        merged_parts.append(fragment)
            
            fragments.extend(merged_parts)
    
    return fragments

def split_text_by_punctuation(text, min_length=10, max_length=50):
    """
    兼容性函數 - 調用新的句子切分函數
    """
    return split_text_to_sentences(text, min_length, max_length)

def process_text_splitting(df, text_column='text', min_length=10, max_length=50):
    """
    處理整個資料集的文本切分 - 句子級別切分
    
    Args:
        df (DataFrame): 原始資料集
        text_column (str): 文本欄位名稱
        min_length (int): 最小句子片段長度
        max_length (int): 最大句子片段長度
    
    Returns:
        DataFrame: 切分後的資料集
    """
    print(f"📊 開始處理文本切分...")
    print(f"  原始資料筆數: {len(df)}")
    print(f"  文本欄位: {text_column}")
    print(f"  句子片段長度範圍: {min_length}-{max_length} 字")
    
    split_data = []
    total_fragments = 0
    processed_texts = 0
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="切分進度"):
        original_text = row[text_column]
        
        # 跳過太短的文本
        if len(original_text) < min_length:
            continue
        
        # 切分文本為句子片段
        fragments = split_text_to_sentences(original_text, min_length, max_length)
        
        # 為每個片段創建新記錄
        for frag_idx, fragment in enumerate(fragments):
            new_row = row.copy()
            new_row[text_column] = fragment
            new_row['original_index'] = idx
            new_row['fragment_index'] = frag_idx
            new_row['original_text_length'] = len(original_text)
            new_row['fragment_length'] = len(fragment)
            new_row['source_type'] = 'sentence_fragment'
            
            split_data.append(new_row)
            total_fragments += 1
        
        processed_texts += 1
    
    # 創建新的DataFrame
    split_df = pd.DataFrame(split_data)
    
    print(f"\n✅ 文本切分完成！")
    print(f"📈 切分統計:")
    print(f"  處理文本數: {processed_texts}")
    print(f"  生成句子片段數: {total_fragments}")
    print(f"  平均每文本片段數: {total_fragments/processed_texts:.1f}")
    
    if not split_df.empty:
        print(f"\n📏 片段長度統計:")
        length_stats = split_df['fragment_length'].describe()
        print(f"  平均長度: {length_stats['mean']:.1f} 字")
        print(f"  最短片段: {length_stats['min']:.0f} 字")
        print(f"  最長片段: {length_stats['max']:.0f} 字")
        print(f"  中位數長度: {length_stats['50%']:.1f} 字")
        
        # 長度分布
        length_ranges = [
            (10, 20, "短片段"),
            (20, 30, "中短片段"),
            (30, 40, "中等片段"),
            (40, 50, "長片段"),
            (50, 100, "超長片段")
        ]
        
        print(f"\n📊 片段長度分布:")
        for min_len, max_len, desc in length_ranges:
            count = len(split_df[(split_df['fragment_length'] >= min_len) & 
                                (split_df['fragment_length'] < max_len)])
            percentage = count / len(split_df) * 100
            print(f"  {desc} ({min_len}-{max_len}字): {count} 個 ({percentage:.1f}%)")
    
    return split_df

# 執行文本切分
if 'dataset_df' in globals() and dataset_df is not None:
    print(f"🎯 對載入的資料集進行句子級別文本切分...")
    
    # 設定切分參數 - 改為句子級別
    MIN_FRAGMENT_LENGTH = 10   # 最小片段長度
    MAX_FRAGMENT_LENGTH = 50   # 最大片段長度
    
    print(f"\n⚙️ 切分參數:")
    print(f"  最小片段長度: {MIN_FRAGMENT_LENGTH} 字")
    print(f"  最大片段長度: {MAX_FRAGMENT_LENGTH} 字")
    print(f"  切分模式: 句子級別")
    
    # 執行切分
    split_dataset_df = process_text_splitting(
        df=dataset_df, 
        text_column='text',
        min_length=MIN_FRAGMENT_LENGTH,
        max_length=MAX_FRAGMENT_LENGTH
    )
    
    if not split_dataset_df.empty:
        # 顯示切分範例
        print(f"\n📝 切分範例:")
        print("="*80)
        
        # 找一個有多個片段的原始文本
        sample_original_idx = split_dataset_df['original_index'].value_counts().index[0]
        sample_fragments = split_dataset_df[split_dataset_df['original_index'] == sample_original_idx]
        
        print(f"原始文本 #{sample_original_idx} 被切分為 {len(sample_fragments)} 個句子片段:")
        print()
        
        # 顯示原始文本
        original_text = dataset_df.iloc[sample_original_idx]['text']
        print(f"原始文本: {original_text[:200]}{'...' if len(original_text) > 200 else ''}")
        print()
        print("切分結果:")
        
        for i, (_, row) in enumerate(sample_fragments.iterrows()):
            fragment = row['text']
            length = row['fragment_length']
            print(f"片段 {i+1} ({length}字): {fragment}")
            print("-" * 60)
        
        # 儲存切分後的資料集
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        split_dir = "split_datasets"
        os.makedirs(split_dir, exist_ok=True)
        
        # 儲存為多種格式
        base_filename = f"{split_dir}/sentence_fragments_{timestamp}"
        
        # CSV 格式
        csv_filename = f"{base_filename}.csv"
        split_dataset_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        
        # JSON 格式
        json_filename = f"{base_filename}.json"
        split_dataset_df.to_json(json_filename, orient='records', force_ascii=False, indent=2)
        
        # Parquet 格式
        parquet_filename = f"{base_filename}.parquet"
        split_dataset_df.to_parquet(parquet_filename, index=False)
        
        print(f"\n💾 句子片段資料集已儲存:")
        print(f"  📄 CSV: {csv_filename}")
        print(f"  📋 JSON: {json_filename}")
        print(f"  📦 Parquet: {parquet_filename}")
        
        # 檔案大小統計
        print(f"\n📁 檔案大小:")
        for filename in [csv_filename, json_filename, parquet_filename]:
            if os.path.exists(filename):
                size_mb = os.path.getsize(filename) / (1024 * 1024)
                print(f"  {os.path.basename(filename)}: {size_mb:.2f} MB")
        
        # 儲存到全域變數
        globals()['split_dataset_df'] = split_dataset_df
        
        print(f"\n🎉 句子級別文本切分處理完成！")
        print(f"📋 變數名稱: split_dataset_df")
        print(f"🎯 句子片段資料集可用於後續的 LLM 處理！")
    
    else:
        print("❌ 切分後沒有產生有效片段，請檢查原始資料")
        split_dataset_df = None

else:
    print("❌ 沒有找到資料集，請先執行 Get Data 部分")
    split_dataset_df = None

print("="*80)

🔪 啟動文本切分處理...
❌ 沒有找到資料集，請先執行 Get Data 部分


## LLM AUG

In [7]:
# 🎯 最終版大陸用語識別與篩選系統 - 使用 Ollama 推論並儲存結果
print("🚀 啟動最終版大陸用語識別系統...")

# 定義大陸特有詞彙庫
mainland_terms = {
    "計算機": ["電腦"], "軟件": ["軟體"], "硬件": ["硬體"], "網絡": ["網路"], 
    "數據": ["資料"], "程序": ["程式"], "信息": ["資訊"], "出租車": ["計程車"],
    "公交車": ["公車"], "地鐵": ["捷運"], "質量": ["品質"], "服務員": ["服務生"],
    "土豆": ["馬鈴薯"], "西紅柿": ["番茄"], "搞定": ["完成"], "挺": ["很"],
    "咋": ["怎麼"], "啥": ["什麼"], "微信": [""], "支付寶": [""], "淘寶": [""]
}

# 大陸語法模式
mainland_patterns = [r"挺.*的", r"蠻.*的", r".*得很", r"咋.*", r"啥.*"]

def analyze_features(text):
    """快速特徵分析"""
    mainland_count = sum(1 for term in mainland_terms if term in text)
    pattern_count = sum(1 for pattern in mainland_patterns if re.search(pattern, text))
    return {
        "mainland_terms": [term for term in mainland_terms if term in text],
        "pattern_matches": pattern_count,
        "authenticity_score": mainland_count + pattern_count
    }

def mainland_score_ollama(text, model="gpt-oss:20b"):
    """使用 Ollama 評分大陸用語特徵"""
    prompt = f"""評估文本的大陸用語特徵，每項0或1分：

文本：{text}

評分標準：
1. 大陸特有詞彙：計算機、軟件、出租車、地鐵等
2. 大陸語法習慣：挺...的、蠻...的、咋樣等  
3. 大陸口語表達：搞定、整、弄等
4. 避免繁體用語：不含電腦、軟體、資料等
5. 整體大陸化程度：綜合評估

請按格式回答：
大陸特有詞彙:0
大陸語法習慣:0
大陸口語表達:0
避免繁體用語:1
整體大陸化程度:0
總分:1"""

    try:
        response = ollama.generate(
            model=model,
            prompt=prompt,
            options={'temperature': 0.1, 'max_tokens': 100}
        )
        
        # 解析回應
        scores = {}
        total = 0
        categories = ["大陸特有詞彙", "大陸語法習慣", "大陸口語表達", "避免繁體用語", "整體大陸化程度"]
        
        for line in response['response'].split('\n'):
            for cat in categories:
                if cat in line:
                    match = re.search(r'[：:]\s*([01])', line)
                    if match:
                        score = int(match.group(1))
                        scores[cat] = score
                        total += score
        
        if len(scores) == 5:
            scores["總分"] = total
            return scores, response['response']
        return None, response['response']
        
    except Exception as e:
        return None, str(e)

def process_dataset(df, text_col='text', sample_size=100, threshold=3):
    """處理資料集進行大陸用語篩選"""
    
    print(f"📊 處理資料集：{len(df)} 筆記錄")
    print(f"📝 文本欄位：{text_col}")
    print(f"🎯 樣本大小：{sample_size}")
    print(f"⚖️ 篩選閾值：{threshold}/5")
    
    # 取樣本
    sample_df = df.sample(n=min(sample_size, len(df)), random_state=42)
    texts = sample_df[text_col].tolist()
    
    # 執行推論
    results = []
    authentic_texts = []
    generic_texts = []
    
    print(f"\n🔄 開始 Ollama 推論...")
    
    for i, text in enumerate(tqdm(texts, desc="推論進度")):
        # 特徵分析
        features = analyze_features(text)
        
        # Ollama 評分
        scores, response = mainland_score_ollama(text)
        
        result = {
            'index': i,
            'text': text,
            'text_length': len(text),
            'features': features,
            'scores': scores,
            'response': response,
            'success': scores is not None
        }
        
        # 分類
        if scores and scores.get("總分", 0) >= threshold:
            authentic_texts.append(result)
            category = "真正大陸用語"
        else:
            generic_texts.append(result)
            category = "通用簡體中文"
        
        result['category'] = category
        results.append(result)
        
        # 顯示進度
        if i % 20 == 0 or i < 3:
            score_str = f"{scores['總分']}/5" if scores else "失敗"
            print(f"  第{i+1}筆: {score_str} - {category}")
        
        time.sleep(0.2)  # 控制請求頻率
    
    return results, authentic_texts, generic_texts

def save_results(results, authentic_texts, generic_texts):
    """儲存篩選結果 - 支援切分資料格式"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. 完整結果
    full_data = []
    for r in results:
        row = {
            'text': r['text'],
            'text_length': r['text_length'],
            'category': r['category'],
            'success': r['success'],
            'authenticity_score': r['features']['authenticity_score'],
            'mainland_terms': ','.join(r['features']['mainland_terms'])
        }
        
        # 添加切分相關欄位（如果存在）
        original_row = available_data.iloc[r['index']]
        if 'source_type' in original_row:
            row['source_type'] = original_row['source_type']
        if 'source' in original_row:
            row['source'] = original_row['source']
        if 'fragment_length' in original_row:
            row['fragment_length'] = original_row['fragment_length']
        if 'augmentation_method' in original_row:
            row['augmentation_method'] = original_row['augmentation_method']
        
        if r['scores']:
            row.update({f'score_{k}': v for k, v in r['scores'].items()})
        full_data.append(row)
    
    full_df = pd.DataFrame(full_data)
    full_file = f"mainland_filtering_complete_{timestamp}.csv"
    full_df.to_csv(full_file, index=False, encoding='utf-8-sig')
    
    # 2. 高質量大陸用語數據（切分格式）
    if authentic_texts:
        authentic_data = []
        for r in authentic_texts:
            original_row = available_data.iloc[r['index']]
            auth_row = {
                'text': r['text'],
                'total_score': r['scores']['總分'],
                'mainland_terms': ','.join(r['features']['mainland_terms']),
                'text_length': r['text_length']
            }
            
            # 保留切分相關欄位
            if 'source_type' in original_row:
                auth_row['source_type'] = original_row['source_type']
            if 'source' in original_row:
                auth_row['source'] = original_row['source']
            if 'fragment_length' in original_row:
                auth_row['fragment_length'] = original_row['fragment_length']
            if 'augmentation_method' in original_row:
                auth_row['augmentation_method'] = original_row['augmentation_method']
            if 'original_idx' in original_row:
                auth_row['original_idx'] = original_row['original_idx']
            if 'fragment_index' in original_row:
                auth_row['fragment_index'] = original_row['fragment_index']
            
            authentic_data.append(auth_row)
        
        auth_df = pd.DataFrame(authentic_data)
        auth_csv = f"authentic_mainland_texts_{timestamp}.csv"
        auth_json = f"authentic_mainland_texts_{timestamp}.json"
        
        auth_df.to_csv(auth_csv, index=False, encoding='utf-8-sig')
        auth_df.to_json(auth_json, orient='records', force_ascii=False, indent=2)
        
        print(f"💾 儲存完成:")
        print(f"  📄 完整結果: {full_file}")
        print(f"  ✅ 高質量句子片段數據: {auth_csv}")
        print(f"  📋 JSON格式: {auth_json}")
        
        # 顯示切分資料統計
        if 'source' in auth_df.columns:
            print(f"\n📊 高質量數據來源分布:")
            print(auth_df['source'].value_counts())
        
        return full_df, auth_df
    
    return full_df, None

# 主要執行流程
print("="*60)

# 檢查可用資料集 (優先使用最終切分句子片段資料集)
available_data = None
text_column = 'text'

if 'final_split_augmented_df' in locals() and final_split_augmented_df is not None:
    available_data = final_split_augmented_df
    source_name = "最終句子片段擴增資料集"
elif 'split_dataset_df' in locals() and split_dataset_df is not None:
    available_data = split_dataset_df
    source_name = "句子片段資料集"
elif 'optimized_augmented_df' in locals() and optimized_augmented_df is not None:
    available_data = optimized_augmented_df
    source_name = "優化擴增資料集"
elif 'dataset_df' in locals() and dataset_df is not None:
    available_data = dataset_df  
    source_name = "原始資料集"

if available_data is not None:
    print(f"✅ 使用 {source_name}，共 {len(available_data)} 筆記錄")
    
    # 執行篩選（可調整參數）
    SAMPLE_SIZE = 50    # 處理樣本數量
    THRESHOLD = 3       # 篩選閾值
    
    print(f"\n🎯 開始執行大陸用語篩選...")
    results, authentic_results, generic_results = process_dataset(
        df=available_data,
        text_col=text_column,
        sample_size=SAMPLE_SIZE,
        threshold=THRESHOLD
    )
    
    # 統計結果
    print(f"\n📊 篩選結果統計:")
    print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
    print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
    print(f"  📈 篩選率: {len(authentic_results)/len(results)*100:.1f}%")
    
    # 顯示範例
    if authentic_results:
        print(f"\n📝 高質量大陸用語範例:")
        for i, r in enumerate(authentic_results[:3]):
            preview = r['text'][:60] + "..." if len(r['text']) > 60 else r['text']
            print(f"  {i+1}. (得分:{r['scores']['總分']}) {preview}")
    
    # 儲存結果
    print(f"\n💾 儲存結果...")
    full_df, auth_df = save_results(results, authentic_results, generic_results)
    
    # 設定全域變數
    globals()['mainland_filtering_results'] = results
    globals()['authentic_mainland_data'] = auth_df
    
    print(f"\n🎉 大陸用語識別與篩選完成！")
    print(f"📋 可用變數: mainland_filtering_results, authentic_mainland_data")
    print(f"🎯 最終輸出為句子級別的片段資料 (10-50字)")
    
else:
    print("❌ 沒有找到可用的資料集")
    print("💡 請先執行前面的資料載入、文本切分或擴增步驟")

print("="*60)

🚀 啟動最終版大陸用語識別系統...
❌ 沒有找到可用的資料集
💡 請先執行前面的資料載入、文本切分或擴增步驟


## 📋 句子級別切分示例

以下是新的句子級別切分功能的演示。原文本會被切分成10-50字的句子片段，適合進行更精細的大陸用語分析。

In [1]:
# 📋 句子級別切分示例演示
print("🔍 句子級別切分功能演示")

# 示例文本（您提到的例子）
example_text = """如今，不管是在大型商超还是巷子里的小铺，随处可见各种扫码支付的身影。而随着扫码支付线下布局的逐渐广泛和已培育成熟的用户消费习惯，使得近期扫码支付风潮颇有横扫支付市场的势头。不管你是商家和个人都可以开通无卡支付！对此，优壹付方面负责人表示："随着费率改革的正式推行，使得无卡支付、扫码支付进入了全国爆发期。目前，扫码支付市场也激战正酣，越来越多的企业、代理商以及个体都在进军移动支付战场抢滩分食。"""

print(f"📝 原始文本 ({len(example_text)}字):")
print(f"{example_text}")
print()

# 執行句子級別切分
fragments = split_text_to_sentences(example_text, min_length=10, max_length=50)

print(f"🔪 切分結果 (共{len(fragments)}個片段):")
print("="*80)

for i, fragment in enumerate(fragments, 1):
    print(f"片段 {i} ({len(fragment)}字): {fragment}")
    print("-" * 60)

print(f"\n📊 切分統計:")
print(f"  原始文本長度: {len(example_text)} 字")
print(f"  生成片段數: {len(fragments)}")
print(f"  平均片段長度: {sum(len(f) for f in fragments) / len(fragments):.1f} 字")
print(f"  最短片段: {min(len(f) for f in fragments)} 字")
print(f"  最長片段: {max(len(f) for f in fragments)} 字")

print(f"\n🎯 現在每個片段都可以獨立進行大陸用語分析！")
print("="*80)

🔍 句子級別切分功能演示
📝 原始文本 (197字):
如今，不管是在大型商超还是巷子里的小铺，随处可见各种扫码支付的身影。而随着扫码支付线下布局的逐渐广泛和已培育成熟的用户消费习惯，使得近期扫码支付风潮颇有横扫支付市场的势头。不管你是商家和个人都可以开通无卡支付！对此，优壹付方面负责人表示："随着费率改革的正式推行，使得无卡支付、扫码支付进入了全国爆发期。目前，扫码支付市场也激战正酣，越来越多的企业、代理商以及个体都在进军移动支付战场抢滩分食。



NameError: name 'split_text_to_sentences' is not defined

## 📁 從大文件挑選資料

從指定的大文件中隨機挑選250筆資料進行評分處理。

In [1]:
# 📁 從大文件中挑選250筆資料進行評分
print("🔍 從大文件中挑選資料...")

import random

# 文件路徑
file_path = "/Users/edwardhuang/Documents/GitHub/LLM_Aug/data/CLUECorpusSmall.txt"
target_samples = 250

print(f"📂 目標文件: {file_path}")
print(f"🎯 目標樣本數: {target_samples}")

def count_lines_in_file(file_path):
    """計算文件總行數"""
    print("📊 正在計算文件總行數...")
    with open(file_path, 'r', encoding='utf-8') as f:
        count = 0
        for line in f:
            count += 1
            if count % 100000 == 0:
                print(f"  已計算: {count:,} 行")
        return count

def sample_lines_from_large_file(file_path, num_samples, total_lines=None):
    """從大文件中隨機挑選指定數量的行"""
    
    # 如果沒有提供總行數，先計算
    if total_lines is None:
        total_lines = count_lines_in_file(file_path)
    
    print(f"📈 文件總行數: {total_lines:,}")
    print(f"🎲 隨機挑選 {num_samples} 行...")
    
    # 生成隨機行號
    if num_samples >= total_lines:
        print("⚠️  目標樣本數大於等於總行數，將返回所有行")
        selected_lines = list(range(total_lines))
    else:
        selected_lines = sorted(random.sample(range(total_lines), num_samples))
    
    print(f"✅ 已生成 {len(selected_lines)} 個隨機行號")
    
    # 讀取選中的行
    selected_data = []
    current_line = 0
    next_target_idx = 0
    
    print("📖 正在讀取選中的行...")
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if next_target_idx < len(selected_lines) and current_line == selected_lines[next_target_idx]:
                # 清理文本
                clean_text = line.strip()
                if clean_text:  # 只保留非空行
                    selected_data.append({
                        'line_number': current_line,
                        'text': clean_text,
                        'text_length': len(clean_text)
                    })
                next_target_idx += 1
                
                # 顯示進度
                if len(selected_data) % 50 == 0:
                    print(f"  已挑選: {len(selected_data)} 筆")
                
                # 如果已經找到所有目標行，可以提前結束
                if next_target_idx >= len(selected_lines):
                    break
            
            current_line += 1
    
    return selected_data

try:
    # 設定隨機種子以確保可重現性
    random.seed(42)
    
    # 挑選資料
    selected_data = sample_lines_from_large_file(file_path, target_samples)
    
    print(f"\n✅ 資料挑選完成！")
    print(f"📊 成功挑選: {len(selected_data)} 筆資料")
    
    if selected_data:
        # 轉換為DataFrame
        selected_df = pd.DataFrame(selected_data)
        
        # 顯示基本統計
        print(f"\n📈 文本長度統計:")
        length_stats = selected_df['text_length'].describe()
        for stat_name, value in length_stats.items():
            print(f"  {stat_name}: {value:.1f}")
        
        # 顯示前幾筆範例
        print(f"\n📝 前5筆資料範例:")
        for i in range(min(5, len(selected_df))):
            row = selected_df.iloc[i]
            text = row['text']
            preview = text[:100] + "..." if len(text) > 100 else text
            print(f"第{i+1}筆 (行號:{row['line_number']}, {row['text_length']}字): {preview}")
            print("-" * 80)
        
        # 儲存挑選的資料
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        save_dir = "selected_data"
        os.makedirs(save_dir, exist_ok=True)
        
        # 儲存為多種格式
        base_filename = f"{save_dir}/selected_250_samples_{timestamp}"
        
        csv_filename = f"{base_filename}.csv"
        json_filename = f"{base_filename}.json"
        parquet_filename = f"{base_filename}.parquet"
        
        selected_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        selected_df.to_json(json_filename, orient='records', force_ascii=False, indent=2)
        selected_df.to_parquet(parquet_filename, index=False)
        
        print(f"\n💾 資料已儲存:")
        print(f"  📄 CSV: {csv_filename}")
        print(f"  📋 JSON: {json_filename}")
        print(f"  📦 Parquet: {parquet_filename}")
        
        # 檔案大小
        for filename in [csv_filename, json_filename, parquet_filename]:
            if os.path.exists(filename):
                size_mb = os.path.getsize(filename) / (1024 * 1024)
                print(f"    {os.path.basename(filename)}: {size_mb:.2f} MB")
        
        # 設定全域變數
        globals()['selected_large_file_df'] = selected_df
        
        print(f"\n🎉 250筆資料挑選完成！")
        print(f"📋 可用變數: selected_large_file_df")
        print(f"🎯 現在可以使用這些資料進行句子切分和大陸用語評分！")
        
        # 長度分布分析
        print(f"\n📊 文本長度分布:")
        length_ranges = [
            (0, 50, "短文本"),
            (50, 100, "中短文本"),
            (100, 200, "中等文本"),
            (200, 500, "長文本"),
            (500, 1000, "很長文本"),
            (1000, float('inf'), "超長文本")
        ]
        
        for min_len, max_len, desc in length_ranges:
            if max_len == float('inf'):
                count = len(selected_df[selected_df['text_length'] >= min_len])
            else:
                count = len(selected_df[(selected_df['text_length'] >= min_len) & 
                                      (selected_df['text_length'] < max_len)])
            percentage = count / len(selected_df) * 100
            print(f"  {desc} ({min_len}+字): {count} 筆 ({percentage:.1f}%)")
    
    else:
        print("❌ 沒有成功挑選到任何資料")

except Exception as e:
    print(f"❌ 處理過程中發生錯誤: {str(e)}")
    print("💡 請檢查文件路徑是否正確")

print("="*80)

🔍 從大文件中挑選資料...
📂 目標文件: /Users/edwardhuang/Documents/GitHub/LLM_Aug/data/CLUECorpusSmall.txt
🎯 目標樣本數: 250
📊 正在計算文件總行數...
  已計算: 100,000 行
  已計算: 200,000 行
  已計算: 300,000 行
  已計算: 400,000 行
  已計算: 500,000 行
  已計算: 600,000 行
  已計算: 700,000 行
  已計算: 800,000 行
  已計算: 900,000 行
  已計算: 1,000,000 行
  已計算: 1,100,000 行
  已計算: 1,200,000 行
  已計算: 1,300,000 行
  已計算: 1,400,000 行
  已計算: 1,500,000 行
  已計算: 1,600,000 行
  已計算: 1,700,000 行
  已計算: 1,800,000 行
  已計算: 1,900,000 行
  已計算: 2,000,000 行
  已計算: 2,100,000 行
  已計算: 2,200,000 行
  已計算: 2,300,000 行
  已計算: 2,400,000 行
  已計算: 2,500,000 行
  已計算: 2,600,000 行
  已計算: 2,700,000 行
  已計算: 2,800,000 行
  已計算: 2,900,000 行
  已計算: 3,000,000 行
  已計算: 3,100,000 行
  已計算: 3,200,000 行
  已計算: 3,300,000 行
  已計算: 3,400,000 行
  已計算: 3,500,000 行
  已計算: 3,600,000 行
  已計算: 3,700,000 行
  已計算: 3,800,000 行
  已計算: 3,900,000 行
  已計算: 4,000,000 行
  已計算: 4,100,000 行
  已計算: 4,200,000 行
  已計算: 4,300,000 行
  已計算: 4,400,000 行
  已計算: 4,500,000 行
  已計算: 4,600,000 行
  已計算: 4,700,000 行
  已計算

## 🔄 處理挑選的250筆資料

對挑選的資料進行句子級別切分和大陸用語評分。

In [2]:
# 🔄 處理挑選的250筆資料 - 句子切分 + 大陸用語評分
print("🚀 開始處理挑選的250筆資料...")

if 'selected_large_file_df' in globals() and selected_large_file_df is not None:
    print(f"✅ 找到挑選的資料: {len(selected_large_file_df)} 筆")
    
    # Step 1: 句子級別切分
    print(f"\n🔪 步驟1: 執行句子級別切分...")
    
    selected_split_df = process_text_splitting(
        df=selected_large_file_df,
        text_column='text',
        min_length=10,
        max_length=50
    )
    
    if not selected_split_df.empty:
        print(f"\n📊 切分結果:")
        print(f"  原始文本: {len(selected_large_file_df)} 筆")
        print(f"  切分後句子片段: {len(selected_split_df)} 筆")
        print(f"  平均每文本產生: {len(selected_split_df)/len(selected_large_file_df):.1f} 個片段")
        
        # 顯示切分範例
        print(f"\n📝 切分範例 (前3個原始文本):")
        for orig_idx in selected_split_df['original_index'].unique()[:3]:
            fragments = selected_split_df[selected_split_df['original_index'] == orig_idx]
            original_text = selected_large_file_df.iloc[orig_idx]['text']
            
            print(f"\n原始文本 #{orig_idx}: {original_text[:100]}{'...' if len(original_text) > 100 else ''}")
            print(f"切分為 {len(fragments)} 個片段:")
            
            for i, (_, row) in enumerate(fragments.iterrows()):
                print(f"  片段{i+1} ({row['fragment_length']}字): {row['text']}")
            print("-" * 80)
        
        # Step 2: 大陸用語評分
        print(f"\n🎯 步驟2: 執行大陸用語評分...")
        
        # 設定評分參數
        SAMPLE_SIZE = min(100, len(selected_split_df))  # 從切分後的片段中取樣評分
        THRESHOLD = 3  # 評分閾值
        
        print(f"📊 評分參數:")
        print(f"  評分樣本數: {SAMPLE_SIZE}")
        print(f"  篩選閾值: {THRESHOLD}/5")
        
        # 執行評分
        results, authentic_results, generic_results = process_dataset(
            df=selected_split_df,
            text_col='text',
            sample_size=SAMPLE_SIZE,
            threshold=THRESHOLD
        )
        
        # 統計結果
        print(f"\n📈 評分結果統計:")
        print(f"  總評分數: {len(results)}")
        print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
        print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
        if len(results) > 0:
            print(f"  📊 大陸用語比例: {len(authentic_results)/len(results)*100:.1f}%")
        
        # 顯示高質量範例
        if authentic_results:
            print(f"\n🏆 高質量大陸用語片段範例:")
            for i, r in enumerate(authentic_results[:5]):
                print(f"第{i+1}名 (得分:{r['scores']['總分']}/5): {r['text']}")
                if r['features']['mainland_terms']:
                    print(f"  大陸特有詞彙: {', '.join(r['features']['mainland_terms'])}")
                print("-" * 60)
        
        # 儲存最終結果
        print(f"\n💾 儲存最終結果...")
        full_df, auth_df = save_results(results, authentic_results, generic_results)
        
        # 創建完整處理結果摘要
        summary_data = {
            "processing_summary": {
                "original_samples": len(selected_large_file_df),
                "split_fragments": len(selected_split_df),
                "evaluated_fragments": len(results),
                "high_quality_mainland": len(authentic_results),
                "generic_chinese": len(generic_results),
                "mainland_percentage": len(authentic_results)/len(results)*100 if len(results) > 0 else 0
            },
            "fragment_length_stats": {
                "mean": selected_split_df['fragment_length'].mean(),
                "min": selected_split_df['fragment_length'].min(),
                "max": selected_split_df['fragment_length'].max(),
                "median": selected_split_df['fragment_length'].median()
            }
        }
        
        # 儲存處理摘要
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        summary_file = f"processing_summary_{timestamp}.json"
        with open(summary_file, 'w', encoding='utf-8') as f:
            json.dump(summary_data, f, ensure_ascii=False, indent=2)
        
        print(f"📋 處理摘要已儲存: {summary_file}")
        
        # 設定全域變數
        globals()['selected_split_df'] = selected_split_df
        globals()['selected_mainland_results'] = results
        globals()['selected_authentic_data'] = auth_df
        
        print(f"\n🎉 250筆資料完整處理流程完成！")
        print(f"📊 最終統計:")
        print(f"  原始挑選資料: {len(selected_large_file_df)} 筆")
        print(f"  句子切分後: {len(selected_split_df)} 個片段")
        print(f"  大陸用語評分: {len(results)} 個片段")
        print(f"  高質量大陸用語: {len(authentic_results)} 個片段")
        print(f"\n📋 可用變數:")
        print(f"  selected_large_file_df: 原始挑選的250筆資料")
        print(f"  selected_split_df: 句子切分後的片段資料")
        print(f"  selected_mainland_results: 大陸用語評分結果")
        print(f"  selected_authentic_data: 高質量大陸用語片段")
        
    else:
        print("❌ 句子切分失敗，無法進行後續處理")

else:
    print("❌ 沒有找到挑選的資料")
    print("💡 請先執行前面的資料挑選步驟")

print("="*80)

🚀 開始處理挑選的250筆資料...
❌ 沒有找到挑選的資料
💡 請先執行前面的資料挑選步驟


In [8]:
# 🔧 修正資料設定並執行完整處理流程
print("🔧 修正資料設定...")

# 檢查並設定DataFrame
if 'selected_data' in globals() and selected_data is not None:
    print(f"✅ 找到 selected_data: {len(selected_data)} 筆")
    
    # 轉換為DataFrame
    selected_large_file_df = pd.DataFrame(selected_data)
    
    print(f"📊 DataFrame 建立成功: {selected_large_file_df.shape}")
    print(f"📋 欄位: {list(selected_large_file_df.columns)}")
    
    # 顯示前幾筆範例
    print(f"\n📝 前3筆資料:")
    for i in range(min(3, len(selected_large_file_df))):
        row = selected_large_file_df.iloc[i]
        text = row['text']
        preview = text[:80] + "..." if len(text) > 80 else text
        print(f"第{i+1}筆: {preview}")
    
    print(f"\n🔄 開始執行完整處理流程...")
    
    # Step 1: 句子級別切分
    print(f"\n🔪 步驟1: 執行句子級別切分...")
    
    selected_split_df = process_text_splitting(
        df=selected_large_file_df,
        text_column='text',
        min_length=10,
        max_length=50
    )
    
    if not selected_split_df.empty:
        print(f"\n📊 切分結果:")
        print(f"  原始文本: {len(selected_large_file_df)} 筆")
        print(f"  切分後句子片段: {len(selected_split_df)} 筆")
        print(f"  平均每文本產生: {len(selected_split_df)/len(selected_large_file_df):.1f} 個片段")
        
        # 顯示切分範例
        print(f"\n📝 切分範例 (第1個原始文本):")
        first_orig_idx = selected_split_df['original_index'].iloc[0]
        fragments = selected_split_df[selected_split_df['original_index'] == first_orig_idx]
        original_text = selected_large_file_df.iloc[first_orig_idx]['text']
        
        print(f"原始文本: {original_text}")
        print(f"切分為 {len(fragments)} 個片段:")
        
        for i, (_, row) in enumerate(fragments.iterrows()):
            print(f"  片段{i+1} ({row['fragment_length']}字): {row['text']}")
        print("-" * 80)
        
        # Step 2: 大陸用語評分 (取較小的樣本數進行演示)
        print(f"\n🎯 步驟2: 執行大陸用語評分...")
        
        # 設定評分參數 - 減少樣本數以加快處理
        SAMPLE_SIZE = min(30, len(selected_split_df))  # 取30個片段進行評分演示
        THRESHOLD = 3  # 評分閾值
        
        print(f"📊 評分參數:")
        print(f"  評分樣本數: {SAMPLE_SIZE}")
        print(f"  篩選閾值: {THRESHOLD}/5")
        
        # 執行評分
        results, authentic_results, generic_results = process_dataset(
            df=selected_split_df,
            text_col='text',
            sample_size=SAMPLE_SIZE,
            threshold=THRESHOLD
        )
        
        # 統計結果
        print(f"\n📈 評分結果統計:")
        print(f"  總評分數: {len(results)}")
        print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
        print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
        if len(results) > 0:
            print(f"  📊 大陸用語比例: {len(authentic_results)/len(results)*100:.1f}%")
        
        # 顯示高質量範例
        if authentic_results:
            print(f"\n🏆 高質量大陸用語片段範例:")
            for i, r in enumerate(authentic_results[:3]):
                print(f"第{i+1}名 (得分:{r['scores']['總分']}/5): {r['text']}")
                if r['features']['mainland_terms']:
                    print(f"  大陸特有詞彙: {', '.join(r['features']['mainland_terms'])}")
                print("-" * 60)
        
        # 儲存結果
        print(f"\n💾 儲存處理結果...")
        full_df, auth_df = save_results(results, authentic_results, generic_results)
        
        # 設定全域變數
        globals()['selected_large_file_df'] = selected_large_file_df
        globals()['selected_split_df'] = selected_split_df
        globals()['selected_mainland_results'] = results
        globals()['selected_authentic_data'] = auth_df
        
        print(f"\n🎉 250筆資料完整處理流程完成！")
        print(f"📊 最終統計:")
        print(f"  原始挑選資料: {len(selected_large_file_df)} 筆")
        print(f"  句子切分後: {len(selected_split_df)} 個片段")
        print(f"  大陸用語評分: {len(results)} 個片段")
        print(f"  高質量大陸用語: {len(authentic_results)} 個片段")
        
        print(f"\n📋 可用變數:")
        print(f"  selected_large_file_df: 原始挑選的250筆資料")
        print(f"  selected_split_df: 句子切分後的片段資料")
        print(f"  selected_mainland_results: 大陸用語評分結果")
        if auth_df is not None:
            print(f"  selected_authentic_data: 高質量大陸用語片段 ({len(auth_df)} 筆)")
        
    else:
        print("❌ 句子切分失敗，無法進行後續處理")

else:
    print("❌ 沒有找到 selected_data")
    print("💡 請先執行前面的資料挑選步驟")

print("="*80)

🔧 修正資料設定...
✅ 找到 selected_data: 250 筆
📊 DataFrame 建立成功: (250, 3)
📋 欄位: ['line_number', 'text', 'text_length']

📝 前3筆資料:
第1筆: 身为一个单身小攻是什么体验？镜像问题身为一个单身小受是一种怎样的体验？1.看到长得瘦，腿很好看的男生就会一直盯着看，然后就想去摸两把。2.发誓看到喜欢的一定要去...
第2筆: 朋友替偶庆生~~正巧在FOXTOWN吃完饭，路过这家店~~便进去了~~看了之后觉得环境比旁边的夜色好~~感觉不错正好搞活动，订了个套餐可以做个卡座~~不错不错·...
第3筆: 错误太多影响阅读心情

🔄 開始執行完整處理流程...

🔪 步驟1: 執行句子級別切分...
📊 開始處理文本切分...
  原始資料筆數: 250
  文本欄位: text
  句子片段長度範圍: 10-50 字


切分進度: 100%|██████████| 250/250 [00:00<00:00, 263.29it/s]



✅ 文本切分完成！
📈 切分統計:
  處理文本數: 240
  生成句子片段數: 2153
  平均每文本片段數: 9.0

📏 片段長度統計:
  平均長度: 31.4 字
  最短片段: 10 字
  最長片段: 293 字
  中位數長度: 31.0 字

📊 片段長度分布:
  短片段 (10-20字): 514 個 (23.9%)
  中短片段 (20-30字): 509 個 (23.6%)
  中等片段 (30-40字): 516 個 (24.0%)
  長片段 (40-50字): 518 個 (24.1%)
  超長片段 (50-100字): 84 個 (3.9%)

📊 切分結果:
  原始文本: 250 筆
  切分後句子片段: 2153 筆
  平均每文本產生: 8.6 個片段

📝 切分範例 (第1個原始文本):
原始文本: 身为一个单身小攻是什么体验？镜像问题身为一个单身小受是一种怎样的体验？1.看到长得瘦，腿很好看的男生就会一直盯着看，然后就想去摸两把。2.发誓看到喜欢的一定要去要联系方式，但是从来都没勇气。3.长期用手。真的巨想谈恋爱了，但是为啥在成都作为一个攻还会单身，我真的没想通。我也不矮啊也有181。也不胖啊我日，思来想去是太宅了，很多人都以为我是直男。可能真的太直了。
切分為 7 個片段:
  片段1 (14字): 身为一个单身小攻是什么体验？
  片段2 (21字): 镜像问题身为一个单身小受是一种怎样的体验？
  片段3 (32字): 1.看到长得瘦，腿很好看的男生就会一直盯着看，然后就想去摸两把。
  片段4 (28字): 2.发誓看到喜欢的一定要去要联系方式，但是从来都没勇气。
  片段5 (33字): 真的巨想谈恋爱了，但是为啥在成都作为一个攻还会单身，我真的没想通。
  片段6 (11字): 我也不矮啊也有181。
  片段7 (27字): 也不胖啊我日，思来想去是太宅了，很多人都以为我是直男。
--------------------------------------------------------------------------------

🎯 步驟2: 執行大陸用語評分...
📊 評分參數:
  評分樣本數: 30
  篩選閾值: 3/5
📊 處理資料集：2153 筆記錄
📝 文本欄位：text


推論進度:   0%|          | 0/30 [00:00<?, ?it/s]

  第1筆: 1/5 - 通用簡體中文


...

推論進度:  67%|██████▋   | 20/30 [03:25<01:36,  9.68s/it]

  第21筆: 1/5 - 通用簡體中文


推論進度: 100%|██████████| 30/30 [04:54<00:00,  9.83s/it]


📈 評分結果統計:
  總評分數: 30
  ✅ 真正大陸用語: 1 筆
  🗑️ 通用簡體中文: 29 筆
  📊 大陸用語比例: 3.3%

🏆 高質量大陸用語片段範例:
第1名 (得分:3/5): 住在他们家于是品尝了一下他们的自助晚餐，天庭的创意是不错，咋一看也很漂亮，但不可否认有点旧了。
  大陸特有詞彙: 咋
------------------------------------------------------------

💾 儲存處理結果...


AttributeError: 'NoneType' object has no attribute 'iloc'

In [ ]:
# 🎯 簡化版結果總結和儲存
print("🎯 整理和儲存最終結果...")

# 確保有評分結果
if 'results' in locals() and 'authentic_results' in locals():
    print(f"✅ 找到評分結果")
    
    # 創建完整結果DataFrame
    full_results_data = []
    for r in results:
        row = {
            'text': r['text'],
            'text_length': r['text_length'],
            'category': r['category'],
            'success': r['success'],
            'authenticity_score': r['features']['authenticity_score'],
            'mainland_terms': ','.join(r['features']['mainland_terms'])
        }
        
        if r['scores']:
            row.update({f'score_{k}': v for k, v in r['scores'].items()})
        full_results_data.append(row)
    
    full_results_df = pd.DataFrame(full_results_data)
    
    # 創建高質量大陸用語DataFrame
    if authentic_results:
        authentic_data = []
        for r in authentic_results:
            auth_row = {
                'text': r['text'],
                'total_score': r['scores']['總分'],
                'mainland_terms': ','.join(r['features']['mainland_terms']),
                'text_length': r['text_length'],
                'category': r['category']
            }
            
            # 添加詳細評分
            if r['scores']:
                for key, value in r['scores'].items():
                    if key != '總分':
                        auth_row[f'score_{key}'] = value
            
            authentic_data.append(auth_row)
        
        authentic_df = pd.DataFrame(authentic_data)
    else:
        authentic_df = pd.DataFrame()
    
    # 儲存結果
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_dir = "evaluation_results"
    os.makedirs(results_dir, exist_ok=True)
    
    # 儲存完整結果
    full_results_file = f"{results_dir}/full_evaluation_results_{timestamp}.csv"
    full_results_df.to_csv(full_results_file, index=False, encoding='utf-8-sig')
    
    # 儲存高質量結果
    if not authentic_df.empty:
        authentic_file = f"{results_dir}/high_quality_mainland_texts_{timestamp}.csv"
        authentic_df.to_csv(authentic_file, index=False, encoding='utf-8-sig')
        
        authentic_json = f"{results_dir}/high_quality_mainland_texts_{timestamp}.json"
        authentic_df.to_json(authentic_json, orient='records', force_ascii=False, indent=2)
    
    # 創建處理摘要
    summary = {
        "processing_timestamp": timestamp,
        "source_file": "/Users/edwardhuang/Documents/GitHub/LLM_Aug/data/CLUECorpusSmall.txt",
        "original_samples": len(selected_large_file_df),
        "split_fragments": len(selected_split_df),
        "evaluated_fragments": len(results),
        "high_quality_mainland": len(authentic_results),
        "generic_chinese": len(generic_results),
        "mainland_percentage": len(authentic_results)/len(results)*100 if len(results) > 0 else 0,
        "fragment_length_stats": {
            "mean": float(selected_split_df['fragment_length'].mean()),
            "min": int(selected_split_df['fragment_length'].min()),
            "max": int(selected_split_df['fragment_length'].max()),
            "median": float(selected_split_df['fragment_length'].median())
        },
        "evaluation_sample_size": len(results),
        "threshold_used": 3
    }
    
    # 儲存摘要
    summary_file = f"{results_dir}/processing_summary_{timestamp}.json"
    with open(summary_file, 'w', encoding='utf-8') as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)
    
    print(f"\n💾 結果已成功儲存:")
    print(f"  📊 完整評分結果: {full_results_file}")
    if not authentic_df.empty:
        print(f"  🏆 高質量大陸用語: {authentic_file}")
        print(f"  📋 JSON格式: {authentic_json}")
    print(f"  📋 處理摘要: {summary_file}")
    
    # 顯示檔案大小
    print(f"\n📁 檔案大小:")
    for filename in [full_results_file, authentic_file if not authentic_df.empty else None, summary_file]:
        if filename and os.path.exists(filename):
            size_kb = os.path.getsize(filename) / 1024
            print(f"  {os.path.basename(filename)}: {size_kb:.2f} KB")
    
    # 設定全域變數
    globals()['selected_large_file_df'] = selected_large_file_df
    globals()['selected_split_df'] = selected_split_df
    globals()['selected_mainland_results'] = results
    globals()['selected_authentic_data'] = authentic_df
    globals()['evaluation_summary'] = summary
    
    print(f"\n🎉 250筆資料評分處理完成！")
    print(f"\n📊 最終統計摘要:")
    print(f"  📂 原始大文件: CLUECorpusSmall.txt")
    print(f"  🎲 隨機挑選: {summary['original_samples']} 筆資料")
    print(f"  🔪 句子切分: {summary['split_fragments']} 個片段")
    print(f"  🎯 評分處理: {summary['evaluated_fragments']} 個片段")
    print(f"  ✅ 高質量大陸用語: {summary['high_quality_mainland']} 個片段")
    print(f"  📈 大陸用語比例: {summary['mainland_percentage']:.1f}%")
    
    if authentic_results:
        print(f"\n🏆 發現的高質量大陸用語片段:")
        for i, r in enumerate(authentic_results):
            print(f"  {i+1}. (得分 {r['scores']['總分']}/5): {r['text']}")
            if r['features']['mainland_terms']:
                print(f"     大陸特有詞彙: {', '.join(r['features']['mainland_terms'])}")
    
    print(f"\n📋 可用變數:")
    print(f"  selected_large_file_df: 原始250筆資料")
    print(f"  selected_split_df: 切分後的句子片段")
    print(f"  selected_mainland_results: 評分結果")
    print(f"  selected_authentic_data: 高質量大陸用語")
    print(f"  evaluation_summary: 處理摘要")

else:
    print("❌ 沒有找到評分結果")

print("="*80)